In [1]:
import laspy
import numpy as np
from sklearn import linear_model
import open3d as o3d
import matplotlib.pyplot as plt
import os

In [2]:
# Load the LAZ file
in_file = laspy.read("data.laz")

# Get the classification for each point
classifications = in_file.points['classification']

# Filter out all points who are not class 1 (roofs) - Maybe its better just tp downsample rather that extracting class 1
non_ground_indices = classifications == 1
filtered_points = in_file.points[non_ground_indices]

# Extract the X, Y, and Z coordinates from the FILTERED points
x_coords = filtered_points['X']
y_coords = filtered_points['Y']
z_coords = filtered_points['Z']

# Convert them into a 2D numpy array
points_np = np.vstack((x_coords, y_coords, z_coords)).T

In [3]:
print(f'points after taking classification = 1: {points_np.shape}')

points after taking classification = 1: (12358405, 3)


In [4]:

# we want to filter out ground points from the dataset
# we extract only points over a certain height (1.5m)

def pca_ground_filter(points, height_threshold=1500):
    # 1. PCA on Point Cloud
    mean_point = np.mean(points, axis=0)
    centered_points = points - mean_point
    covariance_matrix = np.cov(centered_points.T)
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
    
    # 2. Determine Ground Normal
    # The eigenvector corresponding to the smallest eigenvalue
    ground_normal = eigenvectors[:, np.argmin(eigenvalues)]
    
    # If the normal points upwards, we flip it to ensure consistency
    if ground_normal[2] < 0:
        ground_normal = -ground_normal

    # 3. Determine Ground Points
    distances = np.dot(centered_points, ground_normal)
    ground_height = np.min(distances)
    
    # 4. Filter Points
    above_ground_indices = np.where(distances - ground_height > height_threshold)[0]
    
    return points[above_ground_indices]


filtered_points = pca_ground_filter(points_np)

filtered_points.shape

(10815230, 3)

In [2]:
# Convert to open3d point cloud and save as PCD
#pcd = o3d.geometry.PointCloud()
#pcd.points = o3d.utility.Vector3dVector(filtered_points)

pcd = o3d.io.read_point_cloud(os.path.join("dataPCD.pcd"))
#o3d.io.write_point_cloud("output.pcd", pcd)
#o3d.visualization.draw_geometries([pcd])

num = len(pcd.points)
print("Number of points before downsampling: ", num)

Number of points before downsampling:  22754456


In [3]:
# we downsample the pointscloud
downpcd = pcd.voxel_down_sample(voxel_size=0.25)
num2 = len(downpcd.points)
print("Number of points after downsampling: ", num2)
print(100-(num2/num * 100),"% less points")
#o3d.visualization.draw_geometries([downpcd])

Number of points after downsampling:  11405690
49.87491680750355 % less points


In [4]:
#statistical outlier removal

cld, ind = downpcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.2)



In [1]:
num3 = len(cld.points)
print(f'num of ponts after statistical outlier removal {num3}')
print(100-(num3/num2 * 100),"% less points")
#o3d.visualization.draw_geometries([cld])

NameError: name 'cld' is not defined

In [12]:
def filter_points_below_height(pcd, height_threshold):
    """
    Filters out points in a point cloud that are below a given height.
    Args:
    - pcd: An o3d.geometry.PointCloud object.
    - height_threshold: The height below which points will be removed.
    
    Returns:
    - An o3d.geometry.PointCloud object with the points below height_threshold removed.
    """
    
    # Convert the point cloud to a numpy array
    points = np.asarray(pcd.points)
    
    # Filter out points below the height threshold
    filtered_points = points[points[:, 2] >= height_threshold]
    
    # Create a new point cloud from the filtered points
    filtered_pcd = o3d.geometry.PointCloud()
    filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)
    
    return filtered_pcd


# Filter out points below a height of, for example, 1.5
height_threshold = 25
filtered_pcd = filter_points_below_height(cld, height_threshold)

num4 = len(filtered_pcd.points)
print(f'number of points after height removal: {num4}')
print(100-(num4/num3 * 100),"% less points")

number of points after height removal: 6527021
28.140013673941453 % less points


In [ ]:
#clustering
cld = filtered_pcd

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(cld.cluster_dbscan(eps=2, min_points=50, print_progress=True))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
cld.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([cld])

In [ ]:
# create individual arrays for clusters
individual_point_clouds = []

for cluster_label in range(len(labels):
    
    cluster_indices = np.where(labels == cluster_label)[0]
    
    cluster_points = cld.select_by_index(cluster_indices)
    
    individual_point_clouds.append(cluster_points)

In [ ]:
#Draw first cluster 

firstCld = individual_point_clouds[0].points
first_cld = o3d.geometry.PointCloud(firstCld) #This actually creates the point cloud from the data in the array
o3d.visualization.draw_geometries([first_cld])